In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [2]:
# Directory paths
base_dir = "C:\\Users\\LENOVO\\Desktop\\SCVD"
train_dir = os.path.join(base_dir, "SCVD_converted", "train")
test_dir = os.path.join(base_dir, "SCVD_converted", "test")

In [3]:
# Function to extract frames from videos
def extract_frames_from_video(video_path, target_size=(224, 224), num_frames=10):
    frames = []
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise FileNotFoundError(f"Failed to open video file: {video_path}")
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                raise IOError(f"Failed to read frame {i} from video: {video_path}")
            frame = cv2.resize(frame, target_size)
            frames.append(frame)
        cap.release()
    except Exception as e:
        print(f"Error while extracting frames from video {video_path}: {e}")
    return np.array(frames)


In [4]:
# Define a custom sequence generator
class CustomSequenceGenerator:
    def __init__(self, data, labels, batch_size):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.num_samples = len(data)

    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, self.num_samples)
        batch_data = self.data[start_idx:end_idx]
        batch_labels = self.labels[start_idx:end_idx]
        return batch_data, batch_labels


In [5]:
# Function to load video frames from directory
def load_video_frames_from_dir(directory, target_size=(224, 224), num_frames=10):
    frames = []
    labels = []
    try:
        print(f"Directory: {directory}")
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            print(f"File path: {file_path}")
            if not os.path.isfile(file_path) or not file_name.endswith('.avi'):
                print(f"Skipping non-video file: {file_path}")
                continue
            print(f"Loading frames from video: {file_path}")
            # Extract frames from video
            video_frames = extract_frames_from_video(file_path, target_size=target_size, num_frames=num_frames)
            if len(video_frames) > 0:
                frames.append(video_frames)
                # Assign label based on class name
                class_name = os.path.basename(directory)
                if class_name.lower() == "violence":
                    labels.append(1)
                else:
                    labels.append(0)
            else:
                print(f"Warning: No frames extracted from video {file_path}")
    except Exception as e:
        print(f"Error while loading video frames from directory {directory}: {e}")
    return np.array(frames), to_categorical(labels, num_classes=2)

In [6]:
# Load training and test data
train_frames = []
train_labels = []
test_frames = []
test_labels = []

In [7]:
# Load training data
for class_name in ["Normal", "Violence", "Weaponized"]:
    class_dir = os.path.join(train_dir, class_name)
    if not os.path.isdir(class_dir):
        print(f"Directory not found: {class_dir}")
        continue
    frames, labels = load_video_frames_from_dir(class_dir)
    train_frames.extend(frames)
    train_labels.extend(labels)

Directory: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n001_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n001_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n002_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n002_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n003_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n003_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n004_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n004_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n005_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n045_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n045_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n046_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n046_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n047_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n047_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n048_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n048_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n049_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n049_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_conve

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n090_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n090_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n091_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n091_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n092_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n092_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n093_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n093_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n094_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n094_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_conve

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n134_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n134_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n135_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n135_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n136_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n136_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n137_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n137_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n138_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n138_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_conve

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n178_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n178_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n179_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n179_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n180_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n180_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n181_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n181_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n182_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Normal\n182_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_conve

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v022_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v022_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v023_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v023_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v024_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v024_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v025_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v025_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v026_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v026_converted.avi
File path: C:\Users\LENOVO\Des

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v065_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v065_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v066_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v066_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v067_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v067_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v068_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v068_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v069_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Violence\v069_converted.avi
File path: C:\Users\LENOVO\Des

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w007_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w007_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w008_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w008_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w009_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w009_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w010_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w010_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w011_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w011_converted.avi
File path:

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w049_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w049_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w050_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w050_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w051_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w051_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w052_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w052_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w053_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w053_converted.avi
File path:

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w092_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w092_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w093_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w093_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w094_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w094_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w095_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w095_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w096_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\train\Weaponized\w096_converted.avi
File path:

In [8]:
# Load test data
for class_name in ["Normal", "Violence", "Weaponized"]:
    class_dir = os.path.join(test_dir, class_name)
    if not os.path.isdir(class_dir):
        print(f"Directory not found: {class_dir}")
        continue
    frames, labels = load_video_frames_from_dir(class_dir)
    test_frames.extend(frames)
    test_labels.extend(labels)

if len(train_frames) == 0 or len(test_frames) == 0:
    print("Error: No data loaded. Exiting.")
    exit()

Directory: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n001_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n001_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n002_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n002_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n003_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n003_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n004_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n004_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n005_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\N

File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n045_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n045_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n046_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Normal\t_n046_converted.avi
Directory: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence\t_v001_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence\t_v001_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence\t_v002_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence\t_v002_converted.avi
File path: C:\Users\LENOVO\Desktop\SCVD\SCVD_converted\test\Violence\t_v003_converted.avi
Loading frames from video: C:\Users\LENOVO\Desktop\SCVD\SCVD_conv

In [9]:
# Convert to numpy arrays
train_frames = np.array(train_frames)
train_labels = np.array(train_labels)
test_frames = np.array(test_frames)
test_labels = np.array(test_labels)

# Shuffle training data
indices = np.arange(train_frames.shape[0])
np.random.shuffle(indices)
train_frames = train_frames[indices]
train_labels = train_labels[indices]

# Normalize pixel values
train_frames = train_frames / 255.0
test_frames = test_frames / 255.0

In [10]:
print(train_frames.shape)

(399, 10, 224, 224, 3)


In [16]:
from keras.layers import Reshape

# Flatten the image sequences
train_frames_flat = train_frames.reshape(train_frames.shape[0], train_frames.shape[1], -1)
test_frames_flat = test_frames.reshape(test_frames.shape[0], test_frames.shape[1], -1)

# Define LSTM model architecture
model = Sequential([
    LSTM(64, input_shape=(train_frames_flat.shape[1], train_frames_flat.shape[2])),
    Dense(2, activation='softmax')
])

# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_frames_flat, train_labels, epochs=30, validation_data=(test_frames_flat, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_frames_flat, test_labels)
print("Test Accuracy:", test_acc)



Epoch 1/30
13/13 [==============================] - 210s 7s/step - loss: 1.3045 - accuracy: 0.7469 - val_loss: 0.7056 - val_accuracy: 0.8537
Epoch 2/30
13/13 [==============================] - 51s 4s/step - loss: 1.0921 - accuracy: 0.7519 - val_loss: 0.5785 - val_accuracy: 0.8537
Epoch 3/30
13/13 [==============================] - 36s 3s/step - loss: 0.8690 - accuracy: 0.7519 - val_loss: 0.4764 - val_accuracy: 0.8537
Epoch 4/30
13/13 [==============================] - 26s 2s/step - loss: 0.6951 - accuracy: 0.7519 - val_loss: 0.4195 - val_accuracy: 0.8537
Epoch 5/30
13/13 [==============================] - 19s 1s/step - loss: 0.5873 - accuracy: 0.7519 - val_loss: 0.4263 - val_accuracy: 0.8537
Epoch 6/30
13/13 [==============================] - 19s 1s/step - loss: 0.5650 - accuracy: 0.7519 - val_loss: 0.4585 - val_accuracy: 0.8537
Epoch 7/30
13/13 [==============================] - 19s 1s/step - loss: 0.5628 - accuracy: 0.7519 - val_loss: 0.4626 - val_accuracy: 0.8537
Epoch 8/30
13/13 [=

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict labels for the test data
predictions = model.predict(test_frames_flat)
predicted_labels = np.argmax(predictions, axis=1)

# Convert one-hot encoded test labels back to categorical labels
true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics with zero_division='warn'
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, zero_division='warn')
recall = recall_score(true_labels, predicted_labels, zero_division='warn')
f1 = f1_score(true_labels, predicted_labels, zero_division='warn')
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", conf_matrix)



3/3 [==============================] - 7s 387ms/step


F:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8536585365853658
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
 [[70  0]
 [12  0]]


In [18]:
# Save the model to a file
model.save("anomaly_detection_model.h5")


In [15]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model("anomaly_detection_model.h5")
